# Installation and Suppression warping

In [1]:
import os
import sys
import warnings
import numpy

def warn(*args, **kwargs):
    pass

warnings.warn = warn
warnings.simplefilter(action='ignore', category=FutureWarning)
numpy.seterr(divide = 'ignore') 

sys.path.append(os.path.dirname(os.path.abspath('')))

!pip install -e ..[tf,tensorforce,baselines,ccxt,fbm] -U

ERROR: File "setup.py" not found. Directory cannot be installed in editable mode: /home/moohnam/tensortrade/examples


# Data loading

In [3]:
import pandas as pd
from tensortrade.exchanges.simulated import SimulatedExchange

df = pd.read_csv('../data/Coinbase_BTCUSD_1h.csv', skiprows=1)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %I-%p')
df = df.set_index('date').sort_index()
exchange = SimulatedExchange(data_frame=df, base_instrument='USD', pretransform=True)

exchange.data_frame

,symbol,open,high,low,close,volume_btc,volume
date,,,,,,,
2017-07-01 11:00:00,BTCUSD,2505.56,2513.38,2495.12,2509.17,114.60,287000.32
2017-07-01 12:00:00,BTCUSD,2509.17,2512.87,2484.99,2488.43,157.36,393142.50
2017-07-01 13:00:00,BTCUSD,2488.43,2488.43,2454.40,2454.43,280.28,693254.01
2017-07-01 14:00:00,BTCUSD,2454.43,2473.93,2450.83,2459.35,289.42,712864.80
2017-07-01 15:00:00,BTCUSD,2459.35,2475.00,2450.00,2467.83,276.82,682105.41
2017-07-01 16:00:00,BTCUSD,2467.83,2469.99,2450.00,2460.01,330.01,811494.29
2017-07-01 17:00:00,BTCUSD,2460.01,2460.68,2428.03,2445.99,370.26,904722.49
2017-07-01 18:00:00,BTCUSD,2445.99,2455.86,2414.00,2425.94,536.06,1303397.79
2017-07-01 19:00:00,BTCUSD,2425.94,2437.86,2406.44,2435.99,461.20,1118954.38


# Data preprocessing (normalization)

# Feature pipeline

In [4]:
from tensortrade.features import FeaturePipeline
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features.stationarity import FractionalDifference

normalize_price = MinMaxNormalizer(["open", "high", "low", "close"])
difference_all = FractionalDifference(difference_order=0.6)

feature_pipeline = FeaturePipeline(steps=[normalize_price, difference_all])

exchange.feature_pipeline = feature_pipeline

exchange.data_frame

,symbol,open,high,low,close,volume_btc,volume
date,,,,,,,
2017-07-01 11:00:00,BTCUSD,2505.56,2513.38,2495.12,2509.17,114.60,287000.32
2017-07-01 12:00:00,BTCUSD,2509.17,2512.87,2484.99,2488.43,157.36,393142.50
2017-07-01 13:00:00,BTCUSD,2488.43,2488.43,2454.40,2454.43,280.28,693254.01
2017-07-01 14:00:00,BTCUSD,2454.43,2473.93,2450.83,2459.35,289.42,712864.80
2017-07-01 15:00:00,BTCUSD,2459.35,2475.00,2450.00,2467.83,276.82,682105.41
2017-07-01 16:00:00,BTCUSD,2467.83,2469.99,2450.00,2460.01,330.01,811494.29
2017-07-01 17:00:00,BTCUSD,2460.01,2460.68,2428.03,2445.99,370.26,904722.49
2017-07-01 18:00:00,BTCUSD,2445.99,2455.86,2414.00,2425.94,536.06,1303397.79
2017-07-01 19:00:00,BTCUSD,2425.94,2437.86,2406.44,2435.99,461.20,1118954.38


# Action scheme

In [6]:
from tensortrade.actions import ManagedRiskOrders
from tensortrade.instruments import TradingPair
from tensortrade.instruments import Instrument
#from tensortrade.orders.criteria import 

BTC = Instrument('BTC', 8, 'Bitcoin')
USD = Instrument('USD', 2, 'U.S. Dollar')

action_scheme = ManagedRiskOrders(pairs=TradingPair(BTC, USD), trade_sizes = 10)

# Reward 

In [7]:
from tensortrade.rewards import SimpleProfit

reward_scheme = SimpleProfit()

# Enviornment

In [8]:
from tensortrade.environments import TradingEnvironment
from tensortrade.wallets import Portfolio
from tensortrade.wallets import Wallet
from tensortrade.instruments import Quantity

wallets = [Wallet(exchange=exchange, quantity=Quantity(instrument=USD, size=10000)), 
          Wallet(exchange=exchange, quantity=Quantity(instrument=BTC, size=0))]

portfolio = Portfolio(base_instrument='USD', wallets=wallets)

environment = TradingEnvironment(exchange=exchange,
                                 feature_pipeline=feature_pipeline,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 portfolio=portfolio)

# Agent

In [9]:
from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2

model = PPO2
policy = MlpLnLstmPolicy
params = { "learning_rate": 1e-5, 'nminibatches': 1 }

# Training strategy

In [ ]:
from tensortrade.strategies import StableBaselinesTradingStrategy

strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

performance = strategy.run(steps=10000)

('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>


('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>


('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>


('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>


('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>


...

('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>


('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>


('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>


('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>


('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'USD')
('1224000d-6353-4dd0-b166-c86f2d1e0780', 'BTC')
<Wallet: balance=10000.00 USD, locked=0.00 USD>
<Wallet: balance=0.00000000 BTC, locked=0.00000000 BTC>


# Visualize the result

In [ ]:
%matplotlib inline

performance.net_worth.plot()

# Save and Load the Agent

In [ ]:
strategy.save_agent(path="../agents/ppo_btc_1h")

In [ ]:
new_strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

new_strategy.restore_agent(path="../agents/ppo_btc_1h")

# Strategy Evaluation

In [ ]:
import pandas as pd
from tensortrade.environments import TradingEnvironment
from tensortrade.exchanges.simulated import SimulatedExchange

df = pd.read_csv('../data/Coinbase_BTCUSD_d.csv', skiprows=1)
exchange = SimulatedExchange(data_frame=df,
                             feature_pipeline=feature_pipeline,
                             base_instrument='USD',
                             pretransform=True)

environment = TradingEnvironment(exchange=exchange,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme)

new_strategy.environment = environment

test_performance = new_strategy.run(steps=2000)

In [ ]:
%matplotlib inline

test_performance.net_worth.plot()